In [1]:
# xmlrpc_server.ipynb
import copy
from xmlrpc.server import *
from xmlrpc.client import *
import datetime
import pandas as pd
import pickle


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8010), 
                            requestHandler=RequestHandler)

stats_server = ServerProxy("http://localhost:8011")


def slice_log(action_names=[], 
              action_start_date=datetime.datetime(1, 1, 1, 0, 0).strftime("%Y-%m-%d %H:%M:%S"), 
              action_stop_date=datetime.datetime(9999, 12, 31, 23, 59, 59, 999999).strftime("%Y-%m-%d %H:%M:%S"),
              s_duration=0.0,
              e_duration=float('inf')):
    try:
        df = pd.read_json(stats_server.get_log_df())
        if len(action_names) != 0:
            df = df.loc[df['action_name'].isin(action_names)]
        df = df.loc[df['date'] >= action_start_date]
        df = df.loc[df['date'] <= action_stop_date]
        df = df.loc[df['duration'] >= s_duration]
        df = df.loc[df['duration'] <= e_duration]
        return df.to_json()
    except Exception as e:
        print("WARWING! stats server error!")
        print(e)
        return "{}"
        
server.register_function(slice_log, 'slice_log')

# Тест
def ping():
    
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname):
    
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] == sname)
    if (exist == True):
        return sname + ": "+ "bad_boy"
    else:
        return sname + ": "+ "good_boy"
    
server.register_function(black_list_check, 'black_list_check')
    
def black_list_check_with_date_and_fullname(sname, nname, pname, date):
    
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any((frame['Surname'] + frame['Name']  + frame['Patronym']  + frame['Birth']) == sname + nname + pname + date)
    print(exist)
    if (exist):
        return sname + nname + pname + " with birthday: " + date + " : "+ "bad_boy"
    else:
        return sname + nname + pname + " with birthday: " + date + " : "+ "good_boy"

server.register_function(black_list_check_with_date_and_fullname, 'black_list_check_with_date_and_fullname')

# Бинарная передача данных
def send_back_binary(bin_data):
    
    data = bin_data.data
    return Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

# Инверсия цвета
# На вход изображение RGB размерности (M, N, 3) со значениями 0-255
def send_back_inversion(bin_data):
    
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    isColored = len(img_arr.shape) == 3
    for i in range(height):
        for j in range(width):
            if isColored:
                img_arr[i][j][0] = 255 - img_arr[i][j][0]
                img_arr[i][j][1] = 255 - img_arr[i][j][1]
                img_arr[i][j][2] = 255 - img_arr[i][j][2]
            else:
                img_arr[i][j] = 255 - img_arr[i][j]
    
    pimg = pickle.dumps(img_arr)        
    return Binary(pimg)
server.register_function(send_back_inversion, 'color_inversion')


def binary(val, border):
    if val < border:
        return 0
    else:
        return 255


# Бинаризация изображения по задаваемому порогу 1-255
def binary_img(bin_data, border):
    
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]    
    for i in range(height):
        for j in range(width):
            try:
                img_arr[i][j][0] = binary(img_arr[i][j][0], border)
                img_arr[i][j][1] = binary(img_arr[i][j][1], border)
                img_arr[i][j][2] = binary(img_arr[i][j][2], border)
            except Exception:
                img_arr[i][j] = binary(img_arr[i][j], border)
    
    pimg = pickle.dumps(img_arr)        
    return Binary(pimg)
server.register_function(binary_img, 'binary_img')

# Поворот относительно вертикали
def reverse(bin_data):
    
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]    
    for i in range(height):
        for j in range(width // 2):
            temp = copy.deepcopy(img_arr[i][j])
            img_arr[i][j] = copy.deepcopy(img_arr[i][width - j - 1])
            img_arr[i][width - j - 1] = copy.deepcopy(temp)
    
    pimg = pickle.dumps(img_arr)        
    return Binary(pimg)
server.register_function(reverse, 'reverse')

print("Listening on " + str(server.server_address))

try: 
    server.serve_forever()
    print('That was a nice nap.')
except KeyboardInterrupt:
    server.server_close()
    print('What a rude awakening!')


Listening on ('127.0.0.1', 8010)


C:\Users\pavel\AppData\Local\Temp\ipykernel_17140\64881279.py:25: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(stats_server.get_log_df())
127.0.0.1 - - [30/Nov/2023 20:20:56] "POST /RPC2 HTTP/1.1" 200 -


      id action_name                date     duration
0      1         pow 2023-11-28 00:03:41  2038.251638
1      2        ping 2023-11-28 00:04:29  2015.470028
2      3         now 2023-11-28 00:04:35  2018.880129
3      4        type 2023-11-28 00:04:41  2028.809547
4      5        type 2023-11-28 00:04:47  2028.699398
..   ...         ...                 ...          ...
144  145   slice_log 2023-11-30 00:23:51  4045.158625
145  146   slice_log 2023-11-30 00:24:44  4053.632259
146  147   slice_log 2023-11-30 00:25:05  4070.728064
147  148   slice_log 2023-11-30 00:25:46  4045.776129
148  149   slice_log 2023-11-30 00:25:56  4057.391405

[149 rows x 4 columns]


C:\Users\pavel\AppData\Local\Temp\ipykernel_17140\64881279.py:25: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(stats_server.get_log_df())
127.0.0.1 - - [30/Nov/2023 20:25:55] "POST /RPC2 HTTP/1.1" 200 -


      id action_name                date     duration
0      1         pow 2023-11-28 00:03:41  2038.251638
1      2        ping 2023-11-28 00:04:29  2015.470028
2      3         now 2023-11-28 00:04:35  2018.880129
3      4        type 2023-11-28 00:04:41  2028.809547
4      5        type 2023-11-28 00:04:47  2028.699398
..   ...         ...                 ...          ...
145  146   slice_log 2023-11-30 00:24:44  4053.632259
146  147   slice_log 2023-11-30 00:25:05  4070.728064
147  148   slice_log 2023-11-30 00:25:46  4045.776129
148  149   slice_log 2023-11-30 00:25:56  4057.391405
149  150   slice_log 2023-11-30 20:20:52  4068.198919

[150 rows x 4 columns]


...

      id action_name                date     duration
0      1         pow 2023-11-28 00:03:41  2038.251638
1      2        ping 2023-11-28 00:04:29  2015.470028
2      3         now 2023-11-28 00:04:35  2018.880129
3      4        type 2023-11-28 00:04:41  2028.809547
4      5        type 2023-11-28 00:04:47  2028.699398
..   ...         ...                 ...          ...
502  503   slice_log 2023-11-30 22:38:37  4059.748650
503  504   slice_log 2023-11-30 22:38:45  4061.998367
504  505   slice_log 2023-11-30 22:40:03  4099.406719
505  506   slice_log 2023-11-30 22:40:11  4049.813032
506  507   slice_log 2023-11-30 22:42:03  4057.562113

[507 rows x 4 columns]


C:\Users\pavel\AppData\Local\Temp\ipykernel_17140\64881279.py:25: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(stats_server.get_log_df())
127.0.0.1 - - [30/Nov/2023 22:43:30] "POST /RPC2 HTTP/1.1" 200 -


      id action_name                date     duration
0      1         pow 2023-11-28 00:03:41  2038.251638
1      2        ping 2023-11-28 00:04:29  2015.470028
2      3         now 2023-11-28 00:04:35  2018.880129
3      4        type 2023-11-28 00:04:41  2028.809547
4      5        type 2023-11-28 00:04:47  2028.699398
..   ...         ...                 ...          ...
503  504   slice_log 2023-11-30 22:38:45  4061.998367
504  505   slice_log 2023-11-30 22:40:03  4099.406719
505  506   slice_log 2023-11-30 22:40:11  4049.813032
506  507   slice_log 2023-11-30 22:42:03  4057.562113
507  508   slice_log 2023-11-30 22:42:11  4074.133873

[508 rows x 4 columns]


C:\Users\pavel\AppData\Local\Temp\ipykernel_17140\64881279.py:25: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(stats_server.get_log_df())
127.0.0.1 - - [30/Nov/2023 22:43:39] "POST /RPC2 HTTP/1.1" 200 -


      id action_name                date     duration
0      1         pow 2023-11-28 00:03:41  2038.251638
1      2        ping 2023-11-28 00:04:29  2015.470028
2      3         now 2023-11-28 00:04:35  2018.880129
3      4        type 2023-11-28 00:04:41  2028.809547
4      5        type 2023-11-28 00:04:47  2028.699398
..   ...         ...                 ...          ...
504  505   slice_log 2023-11-30 22:40:03  4099.406719
505  506   slice_log 2023-11-30 22:40:11  4049.813032
506  507   slice_log 2023-11-30 22:42:03  4057.562113
507  508   slice_log 2023-11-30 22:42:11  4074.133873
508  509   slice_log 2023-11-30 22:43:26  4069.511890

[509 rows x 4 columns]
